<a href="https://colab.research.google.com/github/volpter/ML_comment_selection/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, plot_precision_recall_curve
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
main_df = pd.read_csv('./Train_data/labeled.csv', sep =',' )


FileNotFoundError: ignored

In [ ]:
main_df.toxic = main_df.toxic.apply(lambda x: int(x))
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  14412 non-null  object
 1   toxic    14412 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 225.3+ KB


In [ ]:
toxic_df = main_df[main_df.toxic==0]
toxic_df.head()

,comment,toxic
6,В шапке были ссылки на инфу по текущему фильму...,0
12,"Почитайте посты у этого автора,может найдете ч...",0
17,Про графику было обидно) я так то проходил все...,0
28,https: pp.userapi.com c848520 v848520411 11627...,0
35,"Возьмём как пример Россию, западноевропейские ...",0


In [ ]:
train_X, test_X, train_y, test_y = train_test_split(main_df.comment, main_df.toxic, random_state=42, test_size=600)

In [ ]:
stemmer=SnowballStemmer(language='russian')
def preprocess(comment):
    t_com = word_tokenize(comment, language='russian')
    t_com = [t for t in t_com if t not in string.punctuation ]
    t_com = [t for t in t_com if t not in stopwords.words('russian')]
    t_com = [stemmer.stem(i) for i in t_com]
    return (t_com)
        

In [ ]:
vectorizer = TfidfVectorizer(tokenizer = lambda x: preprocess(x))

In [ ]:
train_set = vectorizer.fit_transform(train_X[:2000])

In [ ]:
model = LogisticRegression(random_state=42)
model.fit(train_set[:2000], train_y[:2000])


LogisticRegression(random_state=42)

In [ ]:
model.predict(train_set[:10])

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
list(train_set[0])

[<1x11334 sparse matrix of type '<class 'numpy.float64'>'
 	with 3 stored elements in Compressed Sparse Row format>]